# Generar vectorstore con toda la normativa ARN

In [1]:
# Determina entorno de ejecución: local o Colab

# if (firstrun):
if('google.colab' in str(get_ipython() ) ):
    environment= 'google'
else:
    import os
    if (os.environ.get('PWD')=='/kaggle/working'):
        environment= 'kaggle'
    else:
        environment= 'local'
print(environment)

google


In [2]:
# Descarga de texto normas
if (environment=='google'):
  ! wget https://github.com/bettachini/nlpTP/blob/main/arn/gu%C3%ADas_texto.zip?raw=true -O guías_texto.zip
  ! unzip guías_texto.zip
  ! rm guías_texto.zip

  ! wget https://github.com/bettachini/nlpTP/blob/main/arn/normas_texto.zip?raw=true -O normas_texto.zip
  ! unzip normas_texto.zip
  ! rm normas_texto.zip

--2024-11-23 18:52:47--  https://github.com/bettachini/nlpTP/blob/main/arn/gu%C3%ADas_texto.zip?raw=true
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/bettachini/nlpTP/raw/refs/heads/main/arn/gu%C3%ADas_texto.zip [following]
--2024-11-23 18:52:48--  https://github.com/bettachini/nlpTP/raw/refs/heads/main/arn/gu%C3%ADas_texto.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bettachini/nlpTP/refs/heads/main/arn/gu%C3%ADas_texto.zip [following]
--2024-11-23 18:52:48--  https://raw.githubusercontent.com/bettachini/nlpTP/refs/heads/main/arn/gu%C3%ADas_texto.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent

## 1. Load Text Files from Directory and Extract Document Identifier

To read multiple text files from a directory, load only the base names (ignoring any extra suffix), and assign them as document numbers.

In [1]:
import os
try:
  from langchain_community.document_loaders import TextLoader
  from langchain.schema import Document  # Assuming this is the document class for LangChain
except ImportError:
  ! pip install langchain-community
  from langchain_community.document_loaders import TextLoader
  from langchain.schema import Document  # Assuming this is the document class for LangChain


In [12]:
import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import SnowballStemmer
ss = SnowballStemmer("spanish")

def tildesDiacríticas(texto):
    texto = re.sub("ñ","n",texto)
    texto = re.sub("Ñ","N",texto)
    texto = re.sub("á","a",texto)
    texto = re.sub("é","e",texto)
    texto = re.sub("í","i",texto)
    texto = re.sub("ó","o",texto)
    texto = re.sub("ú","u",texto)
    texto = re.sub("Á","A",texto)
    texto = re.sub("É","E",texto)
    texto = re.sub("Í","I",texto)
    texto = re.sub("Ó","O",texto)
    return re.sub("Ú","U",texto)

# Example text preprocessing function
def preprocess_text(text):
    # tildes y diacríticas
    text = tildesDiacríticas(text)

    # Remove non-alphanumeric characters (except spaces)
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenization (in this case, just splitting by space, but you could use a tokenizer)
    words = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('spanish'))
    words = [word for word in words if word not in stop_words]

    # # Stemming (demasiado violento)
    # words = [ss.stem(word) for word in words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join words back into a single string
    return ' '.join(words)


[nltk_data] Downloading package punkt to
[nltk_data]     /home/vbettachini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/vbettachini/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/vbettachini/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/vbettachini/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [13]:
# Normas

def text_loader_normas(directory_path, documents_list):
  for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
      # Extract the document number from the filename (e.g., 'a-b-c' from 'a-b-c_extra.txt')
      doc_number = 'AR ' + '.'.join(filename.split('_')[0].split('-'))

      # Load the file's content
      loader = TextLoader(os.path.join(directory_path, filename))
      doc_content = loader.load()

      # Preprocess the text
      doc_content[0].page_content = preprocess_text(doc_content[0].page_content)

      # Append document with metadata for vector store
      for page in doc_content:
        documents_list.append(Document(page_content=page.page_content, metadata={"doc_number": doc_number}))

      #documents_list.append({
      #  "content": doc_content[0].page_content,
      #  "metadata": {"doc_number": doc_number}
      #})
  return documents_list

# Define the directory containing your text files
normas_path = './normas_texto'  # Replace with your directory path
normas_lista = []
normas_lista = text_loader_normas(normas_path, normas_lista)

In [14]:
# Guías

def text_loader_guías(directory_path, documents_list):
  for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
      # Extract the document number from the filename (e.g., 'a-b-c' from 'a-b-c_extra.txt')
      # doc_number = 'AR Guía' + '.'.join(filename.split('_')[0].split('-'))
      doc_number = 'Guía AR ' + filename.split('_')[0].split('-')[0][2:]

      # Load the file's content
      loader = TextLoader(os.path.join(directory_path, filename))
      doc_content = loader.load()

      # Preprocess the text
      doc_content[0].page_content = preprocess_text(doc_content[0].page_content)

      # Append document with metadata for vector store
      for page in doc_content:
        documents_list.append(Document(page_content=page.page_content, metadata={"doc_number": doc_number}))

      #documents_list.append({
      #  "content": doc_content[0].page_content,
      #  "metadata": {"doc_number": doc_number}
      #})
  return documents_list

# same but for guías_texto
guías_path = './guías_texto'  # Replace with your directory path
guías_lista = []
guías_lista = text_loader_guías(guías_path, guías_lista)

In [15]:
normativa_lista = normas_lista + guías_lista

In [16]:
guías_lista

[Document(metadata={'doc_number': 'Guía AR 13'}, page_content='guia ar 13 revision 0 almacenamiento residuos radiactivos boletin oficial 171014 autoridad regulatoria nuclear dependiente presidencia nacion republica argentina impresion 2014 aprobada resolucion arn n 53914 autoridad regulatoria nuclear av libertador 8250 c1429bnp ciudad autonoma buenos aire argentina tel 011 63231356 fax 011 632317711798 httpwwwarngovar guia ar 13 revision 0 almacenamiento residuos radiactivos alcance 1 recomendaciones guia aplican almacenamiento residuos radiactivos instalaciones clase i 2 recomendaciones guia aplican almacenamiento in situ residuos radiactivos generados instalaciones minero fabriles 3 recomendaciones guia caracter obligatorio presentadas facilitar cumplimiento norma ar 10121 relativo etapa almacenamiento residuos radiactivos especifican controles relativos seguridad fisica salvaguardias puedan instituirse razones distintas seguridad radiologica impongan tale controles debera cumplirse 

## 2. Split Documents into Chunks (if needed)

Since the code already uses a RecursiveCharacterTextSplitter, you can apply it to the documents list directly:

In [8]:
documents = normativa_lista

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(documents)

## 3. Generate Embeddings and Store in Persistent Vector Store

To save the Chroma vector store to a file, specify a persistent storage path.

In [10]:
try:
  from langchain_ollama import OllamaEmbeddings
except ImportError:
  ! pip install langchain-ollama
  from langchain_ollama import OllamaEmbeddings

# Initialize the embedding model
embeddings_model = OllamaEmbeddings(model="all-minilm")

In [11]:
! pip install chromadb
from langchain.vectorstores import Chroma

In [16]:
# Specify a persistent storage path for Chroma
vectorstore_path = './all-minilm_vectorstore'  # Replace with your path

# Create or load an existing Chroma vector store
try:
  vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings_model,
    persist_directory=vectorstore_path
  )
except:
  #Installs Ollama
  ! curl -fsSL https://ollama.com/install.sh | sh
  # Runs Ollama service in the background
  ! ollama serve &>/dev/null&
  # Downloads the pre-trained model
  ! ollama pull all-minilm

  vectorstore = Chroma.from_documents(
    documents=split_documents,
    embedding=embeddings_model,
    persist_directory=vectorstore_path
  )

# Saves the vector store to disk
vectorstore.persist()

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 797b70c4edf8...   0% ▕▏    0 B/ 45 MB                  pulling manifest 
pulling 797b70c4edf8...   0% ▕▏    0 B/ 45 MB                  pulling man

<ipython-input-16-e7e96a291798>:26: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [19]:
! zip vec.zip -r ./all-minilm_vectorstore/

  adding: all-minilm_vectorstore/ (stored 0%)
  adding: all-minilm_vectorstore/chroma.sqlite3 (deflated 76%)
  adding: all-minilm_vectorstore/de96e232-1d31-4aa1-b558-f58db86ffe9e/ (stored 0%)
  adding: all-minilm_vectorstore/de96e232-1d31-4aa1-b558-f58db86ffe9e/index_metadata.pickle (deflated 38%)
  adding: all-minilm_vectorstore/de96e232-1d31-4aa1-b558-f58db86ffe9e/header.bin (deflated 56%)
  adding: all-minilm_vectorstore/de96e232-1d31-4aa1-b558-f58db86ffe9e/data_level0.bin (deflated 12%)
  adding: all-minilm_vectorstore/de96e232-1d31-4aa1-b558-f58db86ffe9e/link_lists.bin (deflated 87%)
  adding: all-minilm_vectorstore/de96e232-1d31-4aa1-b558-f58db86ffe9e/length.bin (deflated 47%)


In [20]:
!ls

all-minilm_vectorstore	guías_texto  normas_texto  sample_data	vec.zip
